In [1]:
import os
from dotenv import load_dotenv 
import uuid

from sqlalchemy.engine.base import Engine
from sqlalchemy import ForeignKey, create_engine, ForeignKey, String, Integer, Column, text
from sqlalchemy.orm import sessionmaker, declarative_base

SQLAlcehmy starts with `Engine` object which acts as a central source of connection to a database.

- It is globally available and created once for a particular database.
- It is configured with a url string which describes how it should be connected to the database


In [2]:
#tutorial is using sql-lite database but I will be using PostgreSQL
load_dotenv()
conn_str = os.getenv('DATABASE_URL')

`When engine is created in SQLAlchemy, it does not immediately connects with the database`. It is just a factory for creating database connection when they are needed. `The actual connection is established when you perform an operation that requires database interact`


In [3]:
engine: Engine = create_engine(conn_str)

The most basic function that create_engine provides is an access to connection, which can then invoke SQL statements.

- The `Connection` object is how all interaction with database is done


In [4]:
with engine.connect() as connection: #engine.connect() opens a new connection which is accessible as 'connection'
        #using with is handy because it automatically closes the connection when done

        result = connection.execute(text("select 'hello world'")) #executes a simple SQL query that
                #does not interact with any tables. text() indicate that the command is a raw SQL text string
                
        print(result.all()) #fetches all the result of the query and prints it

[('hello world',)]


#### Creating a Table and Inserting data into it


In [12]:
with engine.connect() as conn:
    trans = conn.execute(text("Delete  from some_table"))

In [ ]:
with engine.connect() as connection: #establishes a connection to the database using
        #SQLAlchemy engine which is accessible using 'connection'
        
    trans = connection.begin() #This initiates a new transaction. Transaction is a
                    #sequence of database operations that are executed as a single
                    #unit. Either all operations succeeds (commit) or all operations fail
                    #(rollback).
    try:
        connection.execute(text("CREATE TABLE IF NOT EXISTS some_table(x integer, y integer)"))
        # this sends a SQL command to the database to create a new table (CREATE TABLE) named 
        # some_table with two integers column x and y. The table is only created if it does not 
        # exist already (IF NOT EXISTS)
        
        connection.execute(text("INSERT INTO some_table (x, y) VALUES(:x, :y)"),
                        [{"x": 1, "y": 2}])
        # this sends a SQL command to the database to insert a new row into some_table
        # setting the x column to 1 and the y column to 2. The value is provided in a
        # dictionary. You can execute the above command without text(...) but it is
        # not recommended approach because it will expose to SQL injection attacks
        
        trans.commit() #if the above queries executed successfully, this line commits the
                        # transaction making permanent changes to the database
        
    except Exception as e:
        trans.rollback()  #if any error, this line undoes any change made during the transaction
                        # preserving the previous state of the database
        print(f"An error occurred: {e}")
        raise

In [17]:
from sqlalchemy import Transaction


with engine.connect() as connection:
    transaction: Transaction = connection.begin()
    try:
        connection.execute(text("CREATE TABLE IF NOT EXISTS Users(username TEXT, email TEXT, age Integer)"))
        connection.execute(text("INSERT INTO Users(username, email, age) VALUES(:username, :email, :age)"),
                        [{'username' : 'Maaz Khan', 'email' : 'maazkhan@mail.com', 'age' : 25}])
        transaction.commit()
    except Exception as e:
        transaction.rollback()
        print(f"An error occurred: {e}")
        raise

### Youtube Tutorial


In [2]:
Base = declarative_base()

In [3]:
def generate_uuid():
    return str(uuid.uuid4())

In [4]:
class Users(Base):
    __tablename__ = "users"
    userID = Column("userID", String, primary_key=True, default=generate_uuid)
    firstName = Column('firstName', String)
    lastName = Column('lastName', String)
    profileName = Column('profileName', String)
    email = Column('email', String)
    
    def __init__(self, firstName, lastName, profileName, email):
        self.firstName = firstName
        self.lastName = lastName
        self.profileName = profileName
        self.email = email
        

In [5]:
db = "sqlite:///socialDB.db"
engine = create_engine(db)
Base.metadata.create_all(bind=engine)

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
firstName = "Edward"
lastName = "Clark"
profileName = "Ed7812"
email = "ed@mail.com"

In [8]:
user = Users(firstName, lastName, profileName, email)
session.add(user)
session.commit()
print(user.userID)
print("user added")

a904ec94-a92d-4f5a-86f3-2b9b48d14eff
user added


In [9]:
class Posts(Base):
    __tablename__ = "posts"
    postID = Column("postID", String, primary_key=True, default=generate_uuid)
    postContent = Column("postContent", String)
    userID = Column("userID", String, ForeignKey("users.userID"))

    def __init__(self, userID, postContent):
        self.userID = userID
        self.postContent = postContent

In [10]:
Base.metadata.create_all(bind=engine)

In [11]:
post = Posts(userID=user.userID, postContent='This is a post')  # Use the userID from the user you just added
session.add(post)
session.commit()

print("post added")
print(post.postContent, post.postID)

post added
This is a post ca8fc23a-74b1-4f11-a029-2bbb425c8a86
